In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd 

from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint


In [0]:
import io

max_features = 20000
maxlen = 100
# train = pd.read_csv(io.StringIO(uploaded['train.csv'].decode('utf-8')))
# test = pd.read_csv(io.StringIO(uploaded['test.csv'].decode('utf-8')))

train=pd.read_csv('drive/My Drive/Toxic_comment/train.csv') 
test=pd.read_csv('drive/My Drive/Toxic_comment/test.csv') 
embedding_file = 'drive/My Drive/Toxic_comment/glove.6B.50d.txt'
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
train.head(5)

In [0]:
x_train = train['question_text']
y_train = train['target']
x_test = test['question_text']
print(train.apply(lambda x: sum(x.isnull())))
print(test.apply(lambda x: sum(x.isnull())))
maxlen = len(max(x_train))
print(len(x_train))
print(len(y_train))
print(y_train.unique())

In [0]:
def get_embedding_coefs(word, *arr): 
  return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_embedding_coefs(*o.strip().split()) for o in open(embedding_file,'r', encoding="utf8"))

In [0]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_train = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_test = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

In [0]:
all_embs = np.stack(embeddings_index.values())
emb_mean, emb_std = all_embs.mean(), all_embs.std()
embed_size = 50
i
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [0]:
def get_model():
    embed_size = 128
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size)(inp)
    x = Bidirectional(LSTM(128, return_sequences=True,dropout=0.5,recurrent_dropout=0.1))(x)
    x = GlobalMaxPool1D()(x)
    x = Dropout(0.5)(x)
    x = Dense(50, activation="relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

In [0]:
model = get_model()
batch_size = 32
epochs = 2
model.fit(X_train, y, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Train on 104589 samples, validate on 11622 samples
Epoch 1/10
104589/104589 [==============================] - 1886s 18ms/step - loss: 0.0874 - acc: 0.9728 - val_loss: 0.0495 - val_acc: 0.9824
Epoch 2/10
104589/104589 [==============================] - 1878s 18ms/step - loss: 0.0532 - acc: 0.9812 - val_loss: 0.0489 - val_acc: 0.9825
Epoch 3/10
104589/104589 [==============================] - 1853s 18ms/step - loss: 0.0482 - acc: 0.9823 - val_loss: 0.0487 - val_acc: 0.9829
Epoch 4/10
104589/104589 [==============================] - 1855s 18ms/step - loss: 0.0445 - acc: 0.9834 - val_loss: 0.0516 - val_acc: 0.9825
Epoch 5/10
104589/104589 [==============================] - 1835s 18ms/step - loss: 0.0414 - acc: 0.9842 - val_loss: 0.0536 - val_acc: 0.9823
Epoch 6/10
104589/104589 [==============================] - 1877s 18ms/step - loss: 0.0390 - acc: 0.9850 - val_loss: 0.0601 - val_acc: 0.9817
Epoch 7/10
104589/104589 [==============================] - 1852s 18ms/step - loss: 0.0368 - acc:

In [0]:
y_test = model.predict(X_test)
sample_submission = pd.read_csv('drive/My Drive/Toxic_comment/sample_submission.csv')
sample_submission[list_classes] = y_test
sample_submission.to_csv("baseline.csv", index=False)
files.download('baseline.csv')